# Modelo de Predicción de Precios de Casas

Este notebook implementa un modelo de **Regresión Lineal** para predecir precios de casas.

## ¿Qué es la Regresión Lineal?

La Regresión Lineal es un algoritmo de Machine Learning supervisado que modela la relación entre:
- **Variables independientes (features)**: Características de las casas (tamaño, habitaciones, baños, etc.)
- **Variable dependiente (target)**: El precio de la casa

La fórmula general es:
```
Precio = β₀ + β₁×feature₁ + β₂×feature₂ + ... + βₙ×featureₙ
```

Donde:
- **β₀** (intercepto): Valor base del precio
- **β₁, β₂, ..., βₙ** (coeficientes): Peso de cada característica en el precio final

## Flujo de trabajo

1. **Dividir datos**: Separar en entrenamiento (80%) y prueba (20%)
2. **Entrenar modelo**: Ajustar el modelo con los datos de entrenamiento
3. **Evaluar modelo**: Medir qué tan bien predice con datos de prueba
4. **Guardar modelo**: Persistir el modelo para uso futuro
5. **Hacer predicciones**: Usar el modelo para predecir precios de nuevas casas

## 1. Importar librerías

Importamos todas las librerías necesarias para:
- **numpy**: Cálculos numéricos
- **pandas**: Manipulación de datos
- **sklearn**: Algoritmos de Machine Learning
- **pickle**: Guardar y cargar modelos entrenados

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pickle
import sys
import os

# Añadir el directorio padre al path para poder importar config
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from config import TEST_SIZE, RANDOM_STATE, MODEL_FILE

## 2. Función: División de datos

### ¿Por qué dividir los datos?

Es fundamental dividir el dataset en dos conjuntos:

- **Conjunto de entrenamiento (80%)**: Para que el modelo aprenda los patrones
- **Conjunto de prueba (20%)**: Para evaluar qué tan bien generaliza a datos nuevos

Si evaluáramos con los mismos datos de entrenamiento, el modelo podría estar "memorizando" en lugar de "aprendiendo" (overfitting).

### Parámetros importantes:
- `test_size`: Proporción de datos para prueba (0.2 = 20%)
- `random_state`: Semilla para reproducibilidad (siempre genera la misma división)

In [2]:
def split_data(X, y):
    """Divide datos en entrenamiento y prueba"""
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE
    )
    print(f"\n=== DIVISIÓN DE DATOS ===")
    print(f"Entrenamiento: {X_train.shape[0]} casas")
    print(f"Prueba: {X_test.shape[0]} casas")
    
    return X_train, X_test, y_train, y_test

## 3. Función: Entrenamiento del modelo

### ¿Qué hace esta función?

1. **Crea el modelo**: Instancia un objeto LinearRegression()
2. **Entrena (fit)**: El modelo ajusta sus coeficientes para minimizar el error
3. **Muestra coeficientes**: Cada coeficiente indica cuánto impacta esa característica en el precio

### Interpretación de coeficientes:

Por ejemplo, si el coeficiente de `tamano_m2` es **1500**, significa:
- Por cada metro cuadrado adicional, el precio aumenta $1,500

Si el coeficiente de `edad_anos` es **-2000**, significa:
- Por cada año de antigüedad, el precio disminuye $2,000

El **intercepto** es el precio base cuando todas las características son 0.

In [3]:
def train_model(X_train, y_train):
    """Entrena el modelo de Regresión Lineal"""
    print("\n=== ENTRENANDO MODELO ===")
    modelo = LinearRegression()
    modelo.fit(X_train, y_train)
    print("✓ Modelo entrenado exitosamente")
    
    # Mostrar coeficientes
    print("\nCoeficientes aprendidos:")
    for feature, coef in zip(X_train.columns, modelo.coef_):
        print(f"  {feature}: {coef:.2f}")
    print(f"\nIntercepto: {modelo.intercept_:.2f}")
    
    return modelo

## 4. Función: Evaluación del modelo

### Métricas de evaluación

Esta función calcula 3 métricas principales para medir la calidad del modelo:

#### 1. **MAE (Mean Absolute Error)** - Error Absoluto Medio
- Promedio de las diferencias absolutas entre valores reales y predichos
- **Interpretación**: En promedio, el modelo se equivoca por $X
- **Ejemplo**: MAE = $15,000 → en promedio, nos equivocamos por $15,000

#### 2. **RMSE (Root Mean Squared Error)** - Raíz del Error Cuadrático Medio
- Similar a MAE pero penaliza más los errores grandes
- **Interpretación**: Desviación típica de los errores
- Si RMSE >> MAE, hay algunos errores muy grandes (outliers)

#### 3. **R² (Coeficiente de Determinación)**
- Mide qué porcentaje de la variabilidad del precio es explicado por el modelo
- **Rango**: 0 a 1 (0% a 100%)
- **R² = 0.85** → El modelo explica el 85% de la variación en los precios
- **R² = 0.50** → El modelo explica solo el 50% (modelo débil)
- **R² = 0.95** → Excelente ajuste

### Comparación Real vs Predicción

La función también muestra una tabla con:
- **Real**: Precio real de la casa
- **Predicción**: Precio predicho por el modelo
- **Diferencia**: Qué tan lejos está la predicción del valor real

In [4]:
def evaluate_model(modelo, X_test, y_test):
    """Evalúa el modelo con métricas"""
    print("\n=== EVALUACIÓN DEL MODELO ===")
    
    y_pred = modelo.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"MAE: ${mae:,.2f}")
    print(f"RMSE: ${rmse:,.2f}")
    print(f"R²: {r2:.4f}")
    print(f"\nEl modelo explica el {r2*100:.2f}% de la variación en los precios")
    
    # Comparación detallada
    comparacion = pd.DataFrame({
        'Real': y_test.values,
        'Predicción': y_pred,
        'Diferencia': y_test.values - y_pred
    })
    print("\nComparación Real vs Predicción:")
    print(comparacion)
    
    return y_pred, {'mae': mae, 'rmse': rmse, 'r2': r2}

## 5. Función: Guardar modelo

### ¿Por qué guardar el modelo?

Una vez que entrenamos el modelo, queremos guardarlo para:
- **Evitar re-entrenar**: No necesitamos entrenar de nuevo cada vez
- **Producción**: Usar el modelo en una aplicación o API
- **Compartir**: Otros pueden usar el modelo entrenado

### Pickle

Usamos la librería `pickle` de Python que serializa objetos:
- Convierte el modelo en bytes
- Lo guarda en un archivo `.pkl`
- Puede ser cargado posteriormente con todos sus parámetros intactos

In [5]:
def save_model(modelo):
    """Guarda el modelo entrenado"""
    with open(MODEL_FILE, 'wb') as f:
        pickle.dump(modelo, f)
    print(f"\n✓ Modelo guardado en: {MODEL_FILE}")

## 6. Función: Cargar modelo

### ¿Cuándo cargar un modelo?

Cargamos un modelo guardado cuando:
- Queremos hacer predicciones sin re-entrenar
- Usamos el modelo en producción
- Continuamos trabajando en una nueva sesión

El modelo cargado mantiene:
- Todos los coeficientes aprendidos
- El intercepto
- La capacidad de hacer predicciones inmediatamente

In [6]:
def load_model():
    """Carga el modelo guardado"""
    with open(MODEL_FILE, 'rb') as f:
        modelo = pickle.load(f)
    print(f"✓ Modelo cargado desde: {MODEL_FILE}")
    return modelo

## 7. Ejemplo de uso completo

### Pipeline completo de Machine Learning

A continuación se muestra el flujo típico de un proyecto de ML:

1. **Cargar datos**: Leer el CSV con los datos de casas
2. **Separar X e y**: 
   - X = características (features)
   - y = variable objetivo (target/precio)
3. **Dividir datos**: Train/Test split
4. **Entrenar**: Ajustar el modelo con datos de entrenamiento
5. **Evaluar**: Medir rendimiento con datos de prueba
6. **Guardar**: Persistir el modelo para uso futuro

**Nota**: Descomenta las líneas para ejecutar el pipeline completo con tus datos.

In [7]:
# Ejemplo: Cargar datos (descomentar y ajustar según tus datos)
# from config import RAW_DATA_FILE, FEATURES, TARGET
# df = pd.read_csv(RAW_DATA_FILE)
# X = df[FEATURES]
# y = df[TARGET]

# Dividir datos
# X_train, X_test, y_train, y_test = split_data(X, y)

# Entrenar modelo
# modelo = train_model(X_train, y_train)

# Evaluar modelo
# y_pred, metricas = evaluate_model(modelo, X_test, y_test)

# Guardar modelo
# save_model(modelo)

# Cargar modelo (en otra sesión)
# modelo_cargado = load_model()

## 8. Predicciones con el modelo

### Hacer predicciones en producción

Una vez que el modelo está entrenado y guardado, puedes usarlo para predecir precios de casas nuevas.

### Paso a paso:

1. **Crear un DataFrame** con las características de la nueva casa
2. **Importante**: Debe tener las mismas columnas (en el mismo orden) que los datos de entrenamiento
3. **Usar .predict()** para obtener el precio estimado

### Ejemplo de interpretación:

Si una casa tiene:
- 150 m²
- 3 habitaciones
- 2 baños
- 10 años de antigüedad
- 5 km del centro

El modelo calculará: `Precio = intercepto + (1500 × 150) + (coef_hab × 3) + ...`

Y devolverá un precio estimado, por ejemplo: **$250,000**

In [8]:
# Ejemplo de predicción para una nueva casa
# nueva_casa = pd.DataFrame({
#     'tamano_m2': [150],
#     'habitaciones': [3],
#     'banos': [2],
#     'edad_anos': [10],
#     'distancia_centro_km': [5]
# })

# precio_predicho = modelo.predict(nueva_casa)
# print(f"Precio predicho: ${precio_predicho[0]:,.2f}")